In [ ]:
import torch
from diffusers import AutoencoderKL
from PIL import Image


vae = AutoencoderKL.from_pretrained('/store/CIA/js2164/repos/diffusion/stable-diffusion-v1-5', subfolder="vae")

In [ ]:
def encode_img(input_img):
    # Single image -> single latent in a batch (so size 1, 4, 64, 64)
    if len(input_img.shape)<4:
        input_img = input_img.unsqueeze(0)
    with torch.no_grad():
        latent = vae.encode(input_img*2 - 1) # Note scaling
    return 0.18215 * latent.latent_dist.sample()

def decode_img(latents):
    # bath of latents -> list of images
    latents = (1 / 0.18215) * latents
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach()
    return image

In [ ]:
import nb_utils
import run_lib
from configs.utils import read_config
from lightning_data_modules.SRFLOWDataset import UnpairedDataModule

In [ ]:
config = read_config('configs/celebA/ddpm.py')
data_m = UnpairedDataModule(config)

In [ ]:
data_m.setup()
loader = data_m.train_dataloader()

In [ ]:
data = next(iter(loader))
x=data[0][None, ...]

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x[0].permute(1,2,0))

In [ ]:
z = encode_img(x)
print(z.shape)
x_hat = decode_img(z)
print(x_hat.shape)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(x_hat[0].permute(1,2,0))